# History Validation Methods

## Setups

### Imports

In [2]:
from sqlalchemy import create_engine
from google.cloud import bigquery
import pandas as pd
import requests as rq
from requests import HTTPError
import os


### Init

In [3]:

PG_KIN2_USERNAME = os.environ['PG_KIN_2_USER']
PG_KIN2_PWD = os.environ['PG_KIN_2_PWD']
PG_KIN2_CONNSTR = 'before-migration-blockchain-history.cywwhhj0pzz8.us-east-1.rds.amazonaws.com:5432/kin'
PG_KIN3_USERNAME = os.environ['PG_KIN_3_USER']
PG_KIN3_PWD = os.environ['PG_KIN_3_PWD']
PG_KIN3_CONNSTR = 'kin-blockchain-history.cywwhhj0pzz8.us-east-1.rds.amazonaws.com:5432/kin'

START_DATE = '2019-02-10' #'2019-03-01'
END_DATE = '2019-03-31' #'2019-03-31'


KIN2_ENGINE = create_engine(f'postgresql://{PG_KIN2_USERNAME}:{PG_KIN2_PWD}@{PG_KIN2_CONNSTR}')
KIN3_ENGINE = create_engine(f'postgresql://{PG_KIN3_USERNAME}:{PG_KIN3_PWD}@{PG_KIN3_CONNSTR}')
# Fetch all successful transactinons from BIG QUERY
client = bigquery.Client()

MAX_WALLETS_TO_TEST = 50

/Users/skulas/Dev/venvs/kin_playground/lib/python3.7/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


### Data Save

### Data Read

#### BI Queries

In [4]:
# BI Queries

def run_query_on_bq(query_str):
    query_job = client.query(query_str)
    results = query_job.result()  # Waits for job to complete.
    return results


def df_from_K2_HC(query_str, index_col=None):
    if index_col:
        dataframe = pd.read_sql_query(query_str, con=KIN2_ENGINE, index_col=index_col)
    else:
        dataframe = pd.read_sql_query(query_str, con=KIN2_ENGINE)
    
    return dataframe


def df_from_K3_HC(query_str, index_col=None):
    if index_col:
        dataframe = pd.read_sql_query(query_str, con=KIN3_ENGINE, index_col=index_col)
    else:
        dataframe = pd.read_sql_query(query_str, con=KIN3_ENGINE)
    
    return dataframe


def get_results_as_df(query_str):
    results = run_query_on_bq(query_str)
    df = results.to_dataframe()

    return df


# Fetch top earning wallets from big query
def top_failing_wallets(day, month, year, least=False):
    """
    year-month-day is the latest activity for the wallets to be tested.
    Wallets active after this date may have extra transactions that are not
    in BQ yet.
    If least then return the least failing wallets
    """
    date_str = f'{year}-{month}-{day}'
    if least:
        order = 'ASC'
    else:
        order = 'DESC'
    query_str = f"""
        select cnt, source as wallets
        from
        (
                SELECT source, count(*) as cnt
                FROM `kin-bi.stellar.blockchain_payments`
                where tx_status <> 'txSUCCESS'
                and source in (
                                SELECT source
                                FROM `kin-bi.stellar.blockchain_payments`
                                group by source
                                having max(date) < '{date_str}'

                            )
                group by source
        ) t
        order by cnt {order}
        LIMIT {MAX_WALLETS_TO_TEST}
    """
    
    df = get_results_as_df(query_str)
    return df

def get_a_lot_of_wallets(day, month, year, max_wallets=MAX_WALLETS_TO_TEST):
    """
    Bring a lot of wallets to randomly pick a few for random testing
    """
    
    date_str = f'{year}-{month}-{day}'
    query_str = f"""
        SELECT wallets
        FROM
        (
                SELECT destination AS wallets
                FROM `kin-bi.stellar.blockchain_payments`
                WHERE tx_status = 'txSUCCESS'
                GROUP BY destination
                HAVING MAX(date) > '{date_str}'
        ) 
        UNION DISTINCT
                (
                SELECT source AS wallets
                FROM `kin-bi.stellar.blockchain_payments`
                WHERE tx_status = 'txSUCCESS'
                GROUP BY source
                HAVING MAX(date) > '{date_str}'
        ) 
        LIMIT {max_wallets}
    """
    
    df = get_results_as_df(query_str)
    return df
    
def top_earning_wallets(day, month, year, least=False):
    """
    year-month-day is the latest activity for the wallets to be tested.
    Wallets active after this date may have extra transactions that are not
    in BQ yet.
    If least is True, return the least earning wallets
    """
    if least:
        order = 'ASC'
    else:
        order = 'DESC'
    date_str = f'{year}-{month}-{day}'
    query_str = f"""
        select cnt, destination as wallets
        from
        (
                SELECT destination, count(*) as cnt
                FROM `kin-bi.stellar.blockchain_payments`
                where tx_status = 'txSUCCESS'
                and destination in (
                                SELECT destination
                                FROM `kin-bi.stellar.blockchain_payments`
                                group by destination
                                having max(date) < '{date_str}'

                            )
                group by destination
        ) t
        order by cnt {order}
        LIMIT {MAX_WALLETS_TO_TEST}
    """
    
    df = get_results_as_df(query_str)
    return df
    
def top_paying_wallets(day, month, year, least=False):
    """
    year-month-day is the latest activity for the wallets to be tested.
    Wallets active after this date may have extra transactions that are not
    in BQ yet.
    If least is True, return the least paying wallets
    """
    if least:
        order = 'ASC'
    else:
        order = 'DESC'

    date_str = f'{year}-{month}-{day}'
    query_str = f"""
        select cnt, source as wallets
        from
        (
                SELECT source, count(*) as cnt
                FROM `kin-bi.stellar.blockchain_payments`
                where tx_status = 'txSUCCESS'
                and source in (
                                SELECT source
                                FROM `kin-bi.stellar.blockchain_payments`
                                group by source
                                having max(date) < '{date_str}'

                            )
                group by source
        ) t
        WHERE cnt > 2
        ORDER BY cnt {order}
        LIMIT {MAX_WALLETS_TO_TEST}
    """
    
    df = get_results_as_df(query_str)
    return df
    
# Fetch wallet balance from the blockchain
def get_wallet_balance(wallet):
    query_str = 'https://horizon-ecosystem.kininfrastructure.com/accounts/{}'.format(wallet)

    try:
        response = rq.get(query_str)

        # If the response was successful, no Exception will be raised
        response.raise_for_status()
    except HTTPError as http_err:
        print(f'HTTP error occurred: {http_err}')
        return -1
    except Exception as err:
        print(f'Other error occurred: {err}')
        return -2
#     else:
#         print('Success!')
    
    # KIN2 Balance in 10e-5 KIN
    data = response.json()
    balances = data["balances"]
    balance_tot = -3
    for balance in balances:
        if balance["asset_type"] != 'native':
            if balance["asset_code"] == 'KIN':
                balance_str = balance["balance"]
                balance_parts = balance_str.split('.')
                balance_in_kin_str = balance_parts[0]
                balance_decimal_str = balance_parts[1]
                
                balance_decimal = int(round(float(balance_decimal_str)/100)) # transform e7 to e5
                balance_tot = int(balance_in_kin_str)*int(1e5)+balance_decimal
    
    if balance_tot < 0:
        print(f'Could not find wallet balance for wallet {wallet}')
    
    return balance_tot
                

# Calculate balance based on history
def calculate_wallet_balance(wallet):
    query_str = f"""
        SELECT (earnin.sum - payout.sum) as tot FROM
        (
          SELECT IFNULL(SUM(amount),0) as sum
          FROM `kin-bi.stellar.blockchain_payments`
          WHERE source = '{wallet}'
          AND tx_status='txSUCCESS'
        )payout,
        (
          SELECT IFNULL(SUM(amount),0) as sum
          FROM `kin-bi.stellar.blockchain_payments`
          WHERE destination = '{wallet}'
          AND tx_status='txSUCCESS'
        )earnin
    """
    result = run_query_on_bq(query_str)
    total = -1
    for row in result:
        total = row.tot

    
    return int(total*1e5)

def get_tx_grouped_by_src_dest():
    # txSUCCESS is always true in this view...
    query_str = f"""
        SELECT source, destination, SUM(amount) as total, COUNT(tx_hash) as tx_num
            FROM `kin-bi.stellar.payments_with_tx_types_view`
            WHERE time < '{END_DATE}'
                AND time > '{START_DATE}'
            GROUP BY source, destination 
    """
    
    df = get_results_as_df(query_str)
    
    return df

#### HC Queries

In [5]:
# HC Queries

def last_XDR_file(kin2=False):
    query_str = """
        SELECT name
        FROM public.lastfile;
    """
    if kin2:
        df = df_from_K2_HC(query_str)
    else:
        df = df_from_K3_HC(query_str)
    
    return df

def txs_and_sums(kin2=False):
    query_str = f"""
    SELECT source, destination, SUM(amount) as total, COUNT(amount) as tx_num, MAX("time") as max_t
        FROM public.payments
        WHERE tx_status='txSUCCESS'
        AND "time" < '{END_DATE}'
        AND "time" > '{START_DATE}'
        GROUP BY source, destination;
    """
    print(f'***WARNING: results from {START_DATE} to {END_DATE} due to delay in XDR dump in KIN2 blockhchain')
    
    if kin2:
        df = df_from_K2_HC(query_str, index_col=['source', 'destination'])
    else:
        df = df_from_K3_HC(query_str, index_col=['source', 'destination'])
        
    return df

### Data Analysis

In [6]:
def check_wallets_balance(wallets, max_wallets=MAX_WALLETS_TO_TEST):
    counter = 1
    rows, _ = wallets.shape
    print(f'Starting to check {rows} wallets ...')
    all_good = True
    for index, row in wallets.iterrows():
        wallet = row['wallets']
        actual_balance = get_wallet_balance(wallet)
        calculated_balance = calculate_wallet_balance(wallet)
        if actual_balance != calculated_balance:
            all_good = False
            print(wallet)
            print(f'{"*"*25} ERROR {"*"*25}')
            print(f'Calculated: {calculated_balance}\nActual: {actual_balance}')
#         else:
#             print(f'{"*"*5} OK {"*"*5}')
        counter += 1
        if counter > max_wallets:
            break
    if all_good:
        print(f'All {counter-1} wallets where OK')
    else:
        print(f'Finished scanning {counter-1} wallets, errors were found')

In [6]:
### Tests

# df = last_XDR_file(kin2=False)
# print(f"{'-'*3} KIN3 {'-'*3}")
# display(df)
# df = last_XDR_file(kin2=True)
# print(f"\n\n\n{'-'*3} KIN2 {'-'*3}")
# display(df)

## Validate KIN2, KIN3 merge

### Reconstruct  merged status from KIN2 HC and KIN3 HC

In [50]:
## Load data from databases
from multiprocessing import Process

kin2 = None
kin3 = None
def fetch_k2():
    kin2 = txs_and_sums(kin2=True)
    kin2.to_pickle(path=f'/Users/skulas/Dev/TEMP/kin2_merge_validation_data__{START_DATE}-{END_DATE}.pkl')
    display(kin2.head(5))

def fetch_k3():
    kin3 = txs_and_sums(kin2=False)
    kin3['total'] = kin3['total']/100000
    kin3.to_pickle(path=f'/Users/skulas/Dev/TEMP/kin3_merge_validation_data__{START_DATE}-{END_DATE}.pkl')
    display(kin3.head(5))


pr1 = Process(target=fetch_k2)
pr1.start()
pr2 = Process(target=fetch_k3)
pr2.start()
pr1.join()
pr2.join()


***WARNING: results from 2019-02-10 to 2019-03-31 due to delay in XDR dump in KIN2 blockhchain
***WARNING: results from 2019-02-10 to 2019-03-31 due to delay in XDR dump in KIN2 blockhchain


,,total,tx_num,max_t
source,destination,,,
GA22KFUU5R7WZQJDKD23YBGJY6QDB74WWAWORPFMJAYSW3QJXXXM6IMN,GBK5HIGM7WYC7M6GYXE6OH7PTGFQNF5A35EHXLGQUWKU6GFOC76LYLWO,100.0,2,2019-03-22 10:47:35
GA22L5L5PEQBV4JC2SQPV5V3FQ3ZNS3H5YBSQQDXOKEUINCRVLYHTEEF,GASNBN5AFBW2WT4U3RUEFBCMKO3LN2BU7RF4AJSVHTFJCRRTDFZOWXRC,10.0,1,2019-03-20 14:01:21
GA22XVC7JGLB46NECKZBPSRG7RGHITBTLE7SCDTEPTZ4LPBCCE5XAVBT,GBK5HIGM7WYC7M6GYXE6OH7PTGFQNF5A35EHXLGQUWKU6GFOC76LYLWO,100.0,2,2019-03-26 13:13:58
GA23DALHY6E3Q4SSH3NWX27S6Z6U34K2CSPCZR7VM4V375DNIJ4GHO3N,GBSKGUAWRPOVBPBGRENOLQIMZAYVKFQDGQXG5MMS7Q7LCKYR4ZDXKNMU,130.0,1,2019-03-15 18:12:42
GA242SIUVEW565Q4R7R5S2CQX5TWTBY6QH3GENK7R77F24WOBFUEATTS,GBK5HIGM7WYC7M6GYXE6OH7PTGFQNF5A35EHXLGQUWKU6GFOC76LYLWO,100.0,2,2019-03-16 04:23:00


total  \
source                                             destination                                                 
GA222AZUVXH3LXX6EJC6L72X33GDXYJFGXPGSSBYBSYIS5W... GCSMRVRSWROT5BYFPPGZEB6CJBPSRX6A4XW3HG7YBXN6B72...  150.0   
GA223G6Y46G2SKXEORCO2W2JJIW3NHTYD36HGOE64ZJG4GH... GAL2UZI6Z2VIHQUGXZKQXPEHYWECJ5GAUHOE7CS2UBGYWU5...   45.0   
                                                   GALYVHCM5IO2FU4VOMYQ7EYU4C3KDILKD2ZTYIZETD6LHX3...   21.0   
                                                   GAMBOGPZMKMVXB2HSOXMMGXKWW7Q77AZKCRRYZDQS77CSU4...   45.0   
                                                   GB3EILGX7SVBSWFAA7HKV37L25Y2LF6W3QG3YKH3VMMHFW6...   10.0   

                                                                                                       tx_num  \
source                                             destination                                                  
GA222AZUVXH3LXX6EJC6L72X33GDXYJFGXPGSSBYBSYIS5W... GCSMRVRSWROT5BYFPPGZEB6CJBPSRX6A4XW3HG7YBXN6B72...       1   
GA223G6Y46G2SKXEORCO2W2JJIW3NHTYD36HGOE64ZJG4GH... GAL2UZI6Z2VIHQUGXZKQXPEHYWECJ5GAUHOE7CS2UBGYWU5...       1   
                                                   GALYVHCM5IO2FU4VOMYQ7EYU4C3KDILKD2ZTYIZETD6LHX3...       3   
                                                   GAMBOGPZMKMVXB2HSOXMMGXKWW7Q77AZKCRRYZDQS77CSU4...       1   
                                                   GB3EILGX7SVBSWFAA7HKV37L25Y2LF6W3QG3YKH3VMMHFW6...       1   

                                                                                                                    max_t  
source                                             destination                                                             
GA222AZUVXH3LXX6EJC6L72X33GDXYJFGXPGSSBYBSYIS5W... GCSMRVRSWROT5BYFPPGZEB6CJBPSRX6A4XW3HG7YBXN6B72... 2019-02-27 08:12:43  
GA223G6Y46G2SKXEORCO2W2JJIW3NHTYD36HGOE64ZJG4GH... GAL2UZI6Z2VIHQUGXZKQXPEHYWECJ5GAUHOE7CS2UBGYWU5... 2019-03-28 06:36:34  
                                                   GALYVHCM5IO2FU4VOMYQ7EYU4C3KDILKD2ZTYIZETD6LHX3... 2019-03-27 19:27:21  
                                                   GAMBOGPZMKMVXB2HSOXMMGXKWW7Q77AZKCRRYZDQS77CSU4... 2019-03-23 16:20:27  
                                                   GB3EILGX7SVBSWFAA7HKV37L25Y2LF6W3QG3YKH3VMMHFW6... 2019-03-27 19:38:48

In [51]:
## Load data from pickles
kin3 = pd.read_pickle(path=f'/Users/skulas/Dev/TEMP/kin3_merge_validation_data__{START_DATE}-{END_DATE}.pkl')
kin2 = pd.read_pickle(path=f'/Users/skulas/Dev/TEMP/kin2_merge_validation_data__{START_DATE}-{END_DATE}.pkl')
display(kin2.head(2))
print(kin2.shape)
display(kin3.head(2))
print(kin3.shape)
# display(kin2.tail(2))
# display(kin3.tail(2))


,,total,tx_num,max_t
source,destination,,,
GA222AZUVXH3LXX6EJC6L72X33GDXYJFGXPGSSBYBSYIS5WOXXLURWNY,GCSMRVRSWROT5BYFPPGZEB6CJBPSRX6A4XW3HG7YBXN6B72E5T2DFNA6,150.0,1,2019-02-27 08:12:43
GA223G6Y46G2SKXEORCO2W2JJIW3NHTYD36HGOE64ZJG4GHW24FBKLAK,GAL2UZI6Z2VIHQUGXZKQXPEHYWECJ5GAUHOE7CS2UBGYWU5QQ3NZYFT6,45.0,1,2019-03-28 06:36:34


(1418110, 3)


,,total,tx_num,max_t
source,destination,,,
GA22KFUU5R7WZQJDKD23YBGJY6QDB74WWAWORPFMJAYSW3QJXXXM6IMN,GBK5HIGM7WYC7M6GYXE6OH7PTGFQNF5A35EHXLGQUWKU6GFOC76LYLWO,100.0,2,2019-03-22 10:47:35
GA22L5L5PEQBV4JC2SQPV5V3FQ3ZNS3H5YBSQQDXOKEUINCRVLYHTEEF,GASNBN5AFBW2WT4U3RUEFBCMKO3LN2BU7RF4AJSVHTFJCRRTDFZOWXRC,10.0,1,2019-03-20 14:01:21


(53129, 3)


In [52]:
## Merge HC2 with HC3
# print(kin3.shape)
# display(kin3.head(3))
# print(kin2.shape)
# display(kin2.head(3))
# hc_merge = pd.merge(kin2, kin3, how='outer', left_index=True, right_index=True).sum(axis=1)
hc_out_merge = pd.merge(kin2, kin3, how='outer', left_index=True, right_index=True).drop(['max_t_x', 'max_t_y'], axis=1)
hc_merge = pd.merge(kin2, kin3, how='inner', left_index=True, right_index=True).drop(['max_t_x', 'max_t_y'], axis=1)
# display(kin2[kin2['source']=='GA222AZUVXH3LXX6EJC6L72X33GDXYJFGXPGSSBYBSYIS5WOXXLURWNY'][kin2['destination']=='GCSMRVRSWROT5BYFPPGZEB6CJBPSRX6A4XW3HG7YBXN6B72E5T2DFNA6'])
# display(hc_merge.loc[hc_merge['source'] == 'GA222AZUVXH3LXX6EJC6L72X33GDXYJFGXPGSSBYBSYIS5WOXXLURWNY'])
# hc_merge.loc[hc_merge['source'].notnull() & hc_merge['column_name'] <= B]
all_transactions, _  = hc_out_merge.shape

# found_in_k3_and_K3 = hc_merge.loc[pd.notnull(hc_merge['total_x']) & pd.notnull(hc_merge['total_y'])]
number_of_merges, _ = hc_merge.shape # found_in_k3_and_K3.shape
print(f'Found {number_of_merges} transactions from Wallet_i to Wallet_j in both blockchains out of a total of {all_transactions} transactions')



Found 7128 transactions from Wallet_i to Wallet_j in both blockchains out of a total of 1464111 transactions


In [53]:
# Replace all the after merge NaN with zer0s and sum
# hc_merge.fillna(value=0, inplace=True)
# hc_merge['total'] = hc_merge['total_x'] + hc_merge['total_y']
# hc_merge['tx_num'] = hc_merge['tx_num_x'] + hc_merge['tx_num_y']
hc_merge_s = hc_merge[['total_x', 'total_y']].sum(axis=1).to_frame('total')
hc_merge_s['tx_num'] = hc_merge[['tx_num_x', 'tx_num_y']].sum(axis=1)
display(hc_merge.head(2))
display(hc_merge_s.head(2))
hc_merge = hc_merge_s

,,total_x,tx_num_x,total_y,tx_num_y
source,destination,,,,
GA2CZPQFFH6KKOR3A2K6TFU4JDWAD6DQFJVXUZPEHKLRI5A4GZDRHNSO,GDJE5TH7NRJ7WAPM6FGQK4LGXTNB24JLH22PR5ZGUQBFFFRLCMJBCTJQ,100.0,1,500.0,1
GA2EAQORJNIO2AETMXNHHJOS56RD2VI7QYTYD2HAIJE5LW7744Y6BVZT,GBSNER43UUQOYXH6QPSZHJVL3EUZU437NEBSALX2UKBR24LBRZJAEJUP,10.0,1,10.0,1


,,total,tx_num
source,destination,,
GA2CZPQFFH6KKOR3A2K6TFU4JDWAD6DQFJVXUZPEHKLRI5A4GZDRHNSO,GDJE5TH7NRJ7WAPM6FGQK4LGXTNB24JLH22PR5ZGUQBFFFRLCMJBCTJQ,600.0,2
GA2EAQORJNIO2AETMXNHHJOS56RD2VI7QYTYD2HAIJE5LW7744Y6BVZT,GBSNER43UUQOYXH6QPSZHJVL3EUZU437NEBSALX2UKBR24LBRZJAEJUP,20.0,2


In [11]:
## Tests

# test = hc_merge.loc[hc_merge.index['source', :] == 'GDNJRI53DAO63JXQE2COPUDE3B3B6V5GGIIGE5QE4VZQZ5S2EHXHMZ6E']
# test = hc_merge.loc[[("source", "GDNJRI53DAO63JXQE2COPUDE3B3B6V5GGIIGE5QE4VZQZ5S2EHXHMZ6E"), ("destination", :)]]
# test = hc_merge.loc[[("source", "GDNJRI53DAO63JXQE2COPUDE3B3B6V5GGIIGE5QE4VZQZ5S2EHXHMZ6E"), :]]
# test = hc_merge.loc[[("source", "GDNJRI53DAO63JXQE2COPUDE3B3B6V5GGIIGE5QE4VZQZ5S2EHXHMZ6E"), ...]]
# test = hc_merge.loc[["GDNJRI53DAO63JXQE2COPUDE3B3B6V5GGIIGE5QE4VZQZ5S2EHXHMZ6E", ...]]
test = hc_merge.loc["GA2CZPQFFH6KKOR3A2K6TFU4JDWAD6DQFJVXUZPEHKLRI5A4GZDRHNSO", 'GDJE5TH7NRJ7WAPM6FGQK4LGXTNB24JLH22PR5ZGUQBFFFRLCMJBCTJQ']
# test = hc_merge.loc[["GDNJRI53DAO63JXQE2COPUDE3B3B6V5GGIIGE5QE4VZQZ5S2EHXHMZ6E", 'GAUKPMKFGYWUARA7HIO2ISVTA3KTTC3XHOGVJCE6IUE5SFN5JP6GHMBF']]

display(test)

total     600.0
tx_num      2.0
Name: (GA2CZPQFFH6KKOR3A2K6TFU4JDWAD6DQFJVXUZPEHKLRI5A4GZDRHNSO, GDJE5TH7NRJ7WAPM6FGQK4LGXTNB24JLH22PR5ZGUQBFFFRLCMJBCTJQ), dtype: float64

### Query same status in BI

In [54]:
## Load from BI and save to pickle
bi_df = get_tx_grouped_by_src_dest()
bi_df.set_index(['source', 'destination'], inplace=True)
bi_df.to_pickle(path=f'/Users/skulas/Dev/TEMP/bi_merge_validation_data_{START_DATE}-{END_DATE}.pkl')

## Load from pickle
bi_df = pd.read_pickle(path=f'/Users/skulas/Dev/TEMP/bi_merge_validation_data_{START_DATE}-{END_DATE}.pkl')

display(bi_df.head(2))
print(bi_df.shape)

,,total,tx_num
source,destination,,
GDB5ZTCUJAWNCVD2FZ4XJSMGCXOTV2FZ4PTZ32C2KVRX3AHH5A6DPEFV,GA5NIPROAFTVGOVXLI7IDZZJ66BKCGX3TBGZXGJPJEB37KMCGHZMGWKV,4.0,4
GDNJRI53DAO63JXQE2COPUDE3B3B6V5GGIIGE5QE4VZQZ5S2EHXHMZ6E,GCQXFKNWIIX6XG6HDK2OPTDZPKF3XTP3TMRGHQPMEAKO7TQE6DI6MWVX,40.0,3


(1464111, 2)


### Compare BI with HC

In [55]:
# display(hc_merge.head(2))
# hc_data = hc_merge[['total', 'tx_num']]
# display(hc_data.head(2))
# display(bi_df.head(2))

diff_merge = pd.merge(hc_merge, bi_df, how='inner', left_index=True, right_index=True)
display(diff_merge.head(10))
print(diff_merge.shape)

total_x  \
source                                             destination                                                   
GA2CZPQFFH6KKOR3A2K6TFU4JDWAD6DQFJVXUZPEHKLRI5A... GDJE5TH7NRJ7WAPM6FGQK4LGXTNB24JLH22PR5ZGUQBFFFR...    600.0   
GA2EAQORJNIO2AETMXNHHJOS56RD2VI7QYTYD2HAIJE5LW7... GBSNER43UUQOYXH6QPSZHJVL3EUZU437NEBSALX2UKBR24L...     20.0   
                                                   GDJE5TH7NRJ7WAPM6FGQK4LGXTNB24JLH22PR5ZGUQBFFFR...   1300.0   
GA2ELW5MAIWIYJ3KLJ5ILXKMB2QMMXV23J56OZJPRRH46QZ... GDJE5TH7NRJ7WAPM6FGQK4LGXTNB24JLH22PR5ZGUQBFFFR...    700.0   
GA2QB5PPI3PFAIT7JGE77Y4LJCGGAKRJRQEBFFDEKWMTGC4... GDJE5TH7NRJ7WAPM6FGQK4LGXTNB24JLH22PR5ZGUQBFFFR...   1600.0   
GA2RLM3B6N22U6FIA3AVLPZEWKZ7MV3WMNMIYOFN2IHUI5O... GDJE5TH7NRJ7WAPM6FGQK4LGXTNB24JLH22PR5ZGUQBFFFR...   2500.0   
GA2TXDLN2HKTMJKYZG6BMO2F56YUWLYKWRXMNTDAM6TWQWU... GBK5HIGM7WYC7M6GYXE6OH7PTGFQNF5A35EHXLGQUWKU6GF...    600.0   
GA2X4CTKZZJSBABUXNKAZE7EVQP336O77AVKRS4RIHV7ABL... GDJE5TH7NRJ7WAPM6FGQK4LGXTNB24JLH22PR5ZGUQBFFFR...   2100.0   
GA2XN2HUW5RUEN7VLTEWWJAXGGZW4UMAPB5HUU2EDUNJ7LU... GDJE5TH7NRJ7WAPM6FGQK4LGXTNB24JLH22PR5ZGUQBFFFR...   2700.0   
GA2ZXY34DRINOBYQ5MGR646G6UHK45A6ZI5PJHEHROWLO7E... GDJE5TH7NRJ7WAPM6FGQK4LGXTNB24JLH22PR5ZGUQBFFFR...   1000.0   

                                                                                                       tx_num_x  \
source                                             destination                                                    
GA2CZPQFFH6KKOR3A2K6TFU4JDWAD6DQFJVXUZPEHKLRI5A... GDJE5TH7NRJ7WAPM6FGQK4LGXTNB24JLH22PR5ZGUQBFFFR...         2   
GA2EAQORJNIO2AETMXNHHJOS56RD2VI7QYTYD2HAIJE5LW7... GBSNER43UUQOYXH6QPSZHJVL3EUZU437NEBSALX2UKBR24L...         2   
                                                   GDJE5TH7NRJ7WAPM6FGQK4LGXTNB24JLH22PR5ZGUQBFFFR...        13   
GA2ELW5MAIWIYJ3KLJ5ILXKMB2QMMXV23J56OZJPRRH46QZ... GDJE5TH7NRJ7WAPM6FGQK4LGXTNB24JLH22PR5ZGUQBFFFR...         3   
GA2QB5PPI3PFAIT7JGE77Y4LJCGGAKRJRQEBFFDEKWMTGC4... GDJE5TH7NRJ7WAPM6FGQK4LGXTNB24JLH22PR5ZGUQBFFFR...        12   
GA2RLM3B6N22U6FIA3AVLPZEWKZ7MV3WMNMIYOFN2IHUI5O... GDJE5TH7NRJ7WAPM6FGQK4LGXTNB24JLH22PR5ZGUQBFFFR...        25   
GA2TXDLN2HKTMJKYZG6BMO2F56YUWLYKWRXMNTDAM6TWQWU... GBK5HIGM7WYC7M6GYXE6OH7PTGFQNF5A35EHXLGQUWKU6GF...        12   
GA2X4CTKZZJSBABUXNKAZE7EVQP336O77AVKRS4RIHV7ABL... GDJE5TH7NRJ7WAPM6FGQK4LGXTNB24JLH22PR5ZGUQBFFFR...         9   
GA2XN2HUW5RUEN7VLTEWWJAXGGZW4UMAPB5HUU2EDUNJ7LU... GDJE5TH7NRJ7WAPM6FGQK4LGXTNB24JLH22PR5ZGUQBFFFR...         7   
GA2ZXY34DRINOBYQ5MGR646G6UHK45A6ZI5PJHEHROWLO7E... GDJE5TH7NRJ7WAPM6FGQK4LGXTNB24JLH22PR5ZGUQBFFFR...         2   

                                                                                                       total_y  \
source                                             destination                                                   
GA2CZPQFFH6KKOR3A2K6TFU4JDWAD6DQFJVXUZPEHKLRI5A... GDJE5TH7NRJ7WAPM6FGQK4LGXTNB24JLH22PR5ZGUQBFFFR...    600.0   
GA2EAQORJNIO2AETMXNHHJOS56RD2VI7QYTYD2HAIJE5LW7... GBSNER43UUQOYXH6QPSZHJVL3EUZU437NEBSALX2UKBR24L...     20.0   
                                                   GDJE5TH7NRJ7WAPM6FGQK4LGXTNB24JLH22PR5ZGUQBFFFR...   1300.0   
GA2ELW5MAIWIYJ3KLJ5ILXKMB2QMMXV23J56OZJPRRH46QZ... GDJE5TH7NRJ7WAPM6FGQK4LGXTNB24JLH22PR5ZGUQBFFFR...    700.0   
GA2QB5PPI3PFAIT7JGE77Y4LJCGGAKRJRQEBFFDEKWMTGC4... GDJE5TH7NRJ7WAPM6FGQK4LGXTNB24JLH22PR5ZGUQBFFFR...   1600.0   
GA2RLM3B6N22U6FIA3AVLPZEWKZ7MV3WMNMIYOFN2IHUI5O... GDJE5TH7NRJ7WAPM6FGQK4LGXTNB24JLH22PR5ZGUQBFFFR...   2500.0   
GA2TXDLN2HKTMJKYZG6BMO2F56YUWLYKWRXMNTDAM6TWQWU... GBK5HIGM7WYC7M6GYXE6OH7PTGFQNF5A35EHXLGQUWKU6GF...    600.0   
GA2X4CTKZZJSBABUXNKAZE7EVQP336O77AVKRS4RIHV7ABL... GDJE5TH7NRJ7WAPM6FGQK4LGXTNB24JLH22PR5ZGUQBFFFR...   2100.0   
GA2XN2HUW5RUEN7VLTEWWJAXGGZW4UMAPB5HUU2EDUNJ7LU... GDJE5TH7NRJ7WAPM6FGQK4LGXTNB24JLH22PR5ZGUQBFFFR...   2700.0   
GA2ZXY34DRINOBYQ5MGR646G6UHK45A6ZI5PJHEHROWLO7E... GDJE5TH7NRJ7WAPM6FGQK4LGXTNB24JLH22PR5ZGUQBFFFR.

(7128, 4)


In [56]:
## Compare sums
# display(hc_merge.loc['GA223G6Y46G2SKXEORCO2W2JJIW3NHTYD36HGOE64ZJG4GHW24FBKLAK'])
# display(bi_df.loc['GA223G6Y46G2SKXEORCO2W2JJIW3NHTYD36HGOE64ZJG4GHW24FBKLAK'])

diffs_sum = diff_merge.loc[diff_merge['total_x'] != diff_merge['total_y']]
display(diffs_sum.head(20))
print(diffs_sum.shape)

,,total_x,tx_num_x,total_y,tx_num_y
source,destination,,,,


(0, 4)


In [57]:
## Compare number of transactions
diffs_tx_num = diff_merge.loc[diff_merge['tx_num_x'] != diff_merge['tx_num_y']]
# display(diffs_tx_num)
print(diffs_tx_num.shape)

(0, 4)


## Validate most FAILED 

In [22]:
tf_wallets = top_failing_wallets(day=8, month=3, year=2019, least=True)

In [23]:
display(tf_wallets.head(5))


,cnt,wallets
0,1,GDE4KQ46GAGSVMQMJ7ITQU5Y73PT5NQ2626RXWVLDJ7QK5...
1,1,GCEE7RPRESZWKQXWXN4JUFWFXVVX5MJYAOEMVRULHFWD42...
2,1,GAIXQ64NER3IYB3KMG52OLUG3GY67OMQEI762H6OBQN2KI...
3,1,GBYSABHQZ37L2VXJCJRHQYT4LOOL74KU3LE66Z43I7EL7I...
4,1,GDR2KJCPKZRHF4457XFOYMXEOAQ3WQERK2P7LTKPNEI4OF...


In [24]:
check_wallets_balance(tf_wallets)

Starting to check 50 wallets ...
All 50 wallets where OK


In [25]:
tf_wallets = top_failing_wallets(day=8, month=3, year=2019)

In [26]:
display(tf_wallets.head(5))


,cnt,wallets
0,15001,GCI7NAZCQ6ETQGC2MMQKG6KZPZKV52DVCLQG23EOSSIQLR...
1,5070,GAOZ3EZKG3WDUQL4DYPMQDM6Y7R3AAZ2AYMAXPA72CRTKT...
2,388,GBY5PZFDZ6Y25S6YRRZ3CXOAIUWOZ3ADONFY2OYCA7GPQC...
3,344,GDCIL6JBVKU5MNZ6RI5ARDMLE5X36SO5G7BRZMU4VDHNFF...
4,203,GCEOEEJVYKOYF4BUKZ26UTPPJOVNTBTOMUWXZYZ4M2XH4A...


In [27]:
check_wallets_balance(tf_wallets)

Starting to check 50 wallets ...
GCI7NAZCQ6ETQGC2MMQKG6KZPZKV52DVCLQG23EOSSIQLRJRJ7QRQWHX
************************* ERROR *************************
Calculated: 1330862400000
Actual: 1330881400000
GCCPLPXHQRU6FMG7EBAZ3SDH4EU7UFUNVD3TBI3K4T4XUQJRWULKRRGK
************************* ERROR *************************
Calculated: 33696000000
Actual: 33695400000
Finished scanning 50 wallets, errors were found


## Validate most and least EARNING

In [28]:
te_wallets = top_earning_wallets(day=15, month=3, year=2019, least=True)

In [29]:
display(te_wallets.head(5))

,cnt,wallets
0,1,GAULW542MCX5BSSNQISFCSKXGFOCNL4SXK6FBLR5BO4E62...
1,1,GABCEBW6P5VM3YUN3ULX3K6D3T6Q6UFHRN5TSYMUQ2JRED...
2,1,GCIPH6YAWPOSM5JH5O3JPVMC5IKNGT2W5J3EMIF5MVO4WU...
3,1,GAQC5OP5D2SD3DAVQC2XFDPIMG6G76XGJVWI7QNJROCTYI...
4,1,GATDCMJXL7HNSYHEICQSKOVEHRYBG26B2MQVG7FIGO56MK...


In [30]:
check_wallets_balance(te_wallets)

Starting to check 50 wallets ...
All 50 wallets where OK


In [31]:
te_wallets = top_earning_wallets(day=15, month=3, year=2019)

In [32]:
display(te_wallets.head(5))

,cnt,wallets
0,44929,GARGQLP3UOEISB7PY4NGNER62ALCUL3FUUCRF4PMTL7ZBW...
1,35159,GAMZSIJSKWTCIEJC5ZCFU5RNFERGN6RV2GIZHWDBIVDR7R...
2,34513,GAUJB7BUSGBQHGDUR4N6FC35AH3HHN63TYRSLR6M7KBNAK...
3,33198,GDBEPLZXWPE2ETNIY7LVSX4IOHFPVTBDI5OZERTSMRMDI4...
4,29369,GCOJU5Q6WXWGOW762ZHTXNZA6WGLROCZSETZDKRH6YG5BZ...


In [33]:
check_wallets_balance(te_wallets)

Starting to check 50 wallets ...
GAMZSIJSKWTCIEJC5ZCFU5RNFERGN6RV2GIZHWDBIVDR7RRVPRQRLJOI
************************* ERROR *************************
Calculated: 520400000
Actual: 520000000
Finished scanning 50 wallets, errors were found


## Validate most and least PAYING

In [8]:
tp_wallets = top_paying_wallets(day=15, month=3, year=2019, least=False)

In [9]:
check_wallets_balance(tp_wallets)

Starting to check 50 wallets ...
GCI7NAZCQ6ETQGC2MMQKG6KZPZKV52DVCLQG23EOSSIQLRJRJ7QRQWHX
************************* ERROR *************************
Calculated: 1330862400000
Actual: 1330881400000
GD3CJ4ROLGUTOC6MB26J3LZKN2ETZFMNU5ED3CXSREJB74H6DKODQD33
************************* ERROR *************************
Calculated: 1445310600000
Actual: 1445354600000
GDIH7RMIKIWLNBBGXSZHI5PYW3HJDE2TLKCQSGRSPVWTQJGKS4YI6CGS
************************* ERROR *************************
Calculated: 17000000
Actual: 4000000
GBPHY5HEAIY7ZT63O5HH3ZB2PHUDFZHMJ77PWJ7ILQTL66JKDGJT7TUI
************************* ERROR *************************
Calculated: 17746800000
Actual: 17787700000
GCY7OD7OF2Z3QLGSOGEWHMYNOFH3CC23O2FQDQ2YFFRVI5RXFHHEMC7Z
************************* ERROR *************************
Calculated: 489900000
Actual: 489700000
Finished scanning 50 wallets, errors were found


In [36]:
tp_wallets = top_paying_wallets(day=15, month=3, year=2019, least=True)

In [37]:
check_wallets_balance(tp_wallets)

Starting to check 50 wallets ...
All 50 wallets where OK


## Validate Random Wallets

In [4]:
# Fetch all active wallets since march 15th
num_of_wallets = 5234583
tr_wallets = get_a_lot_of_wallets(15, 3, 2019, max_wallets=num_of_wallets)

In [20]:
from random import seed
from random import sample

num_of_wallets, _ = tr_wallets.shape
number_of_wallets_to_test = 150
print(f'Chosing randomly {number_of_wallets_to_test} out of {num_of_wallets} wallets')

# select a random sample without replacement
# seed random number generator
seed(5521)#(num_of_wallets)
# prepare a sequence
sequence = [i for i in range(num_of_wallets)]
# select a subset without replacement
wallet_indices = sample(sequence, number_of_wallets_to_test)

tr_chosen_wallets = tr_wallets.iloc[wallet_indices]
display(tr_chosen_wallets.head(3))
xxx, _ = tr_chosen_wallets.shape
print(xxx)

check_wallets_balance(tr_chosen_wallets, max_wallets=number_of_wallets_to_test)

Chosing randomly 150 out of 1025319 wallets


,wallets
778656,GDBZ5KV2IBEEX3E3SIWOLZYNJL2JFEAV57G3XM4KY7TUET...
723120,GB3MKQEQNJ3L2QLFNIOISWASFADEMK2S47OVC6HYURC73E...
772301,GBLZQHI5YQHFQNWDYXZJ7W7WVBMUEWHTYPCV3B777O2X5I...


150
Starting to check 150 wallets ...
GC6LF7MQNBP3MKFJ7FBLJDB4KII3YUDKEJD7HISK73CVHLVNDZAU7TFW
************************* ERROR *************************
Calculated: 6400000
Actual: 4500000
Finished scanning 259035 wallets, errors were found


# NOT SURE ___ DELETE

In [ ]:
engine = create_engine('postgresql://skulas:@localhost:5432/KRE_Fraud_Data')

In [ ]:
# Translate time stamp
res['time'] = res['time'].astype(pd.Timestamp)

In [ ]:
# Dump all data into a local DB for faster queries
res.to_sql('tx_history', engine)